In [1]:
# coding=utf-8
import json

import config
import requests
import fb_auth_token

In [2]:
headers = {
    'app_version': '6.9.4',
    'platform': 'ios',
    "content-type": "application/json",
    "User-agent": "Tinder/7.5.3 (iPhone; iOS 10.3.2; Scale/2.00)",
    "Accept": "application/json"
}

In [12]:
# Получаем access token
fb_access_token = config.fb_access_token

# Получаем user_id
fb_user_id = config.fb_user_id

In [9]:
def get_auth_token(fb_auth_token, fb_user_id):
    if "error" in fb_auth_token:
        return {"error": "could not retrieve fb_auth_token"}
    if "error" in fb_user_id:
        return {"error": "could not retrieve fb_user_id"}
    url = url = 'https://api.gotinder.com/v2/auth/login/facebook'
    req = requests.post(url,
                        headers=headers,
                        data=json.dumps(
                            {'token': fb_auth_token, 'facebook_id': fb_user_id})
                        )
    print('req',req)
    
    #try:
    
    tinder_auth_token = req.json()['data']['api_token']
    print('token :', tinder_auth_token)
    headers.update({"X-Auth-Token": tinder_auth_token})
    print("You have been successfully authorized!")
    return tinder_auth_token

# Запускаем, чтобы получить token
get_auth_token(fb_access_token, fb_user_id)

req <Response [200]>
token : d085480a-53a5-4faf-84ad-c495e7215a9b
You have been successfully authorized!


'd085480a-53a5-4faf-84ad-c495e7215a9b'

In [13]:
# Получаем рекомендации 

def get_recommendations():
    '''
    Returns a list of users that you can swipe on
    '''
    try:
        r = requests.get('https://api.gotinder.com/user/recs', headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong with getting recomendations:", e)

recommendations = get_recommendations()

In [14]:
recommendations

{'status': 200,
 'results': [{'group_matched': False,
   'distance_mi': 1,
   'content_hash': 'JYXCp2sjLUgFjdT4AhGqs49U5ohqQt5DF9jCMSlQugsm2',
   'common_friends': [],
   'common_likes': [],
   'common_friend_count': 0,
   'common_like_count': 0,
   'connection_count': 0,
   '_id': '5ca4c8f1e9a26316002be8be',
   'bio': '173 рост, 55 кг, зеленые глаза, Рыженькая натурэль, веселая, энергичная👻☀️ люблю петь, танцевать, ездить куда-то менять обстановку, спонтанная, заботливая, домашняя',
   'birth_date': '1988-04-24T14:47:42.182Z',
   'name': 'Танюшка',
   'ping_time': '2014-12-09T00:00:00.000Z',
   'photos': [{'id': 'e03a9f1f-6963-4fd6-88bf-5e3c05911eed',
     'crop_info': {'processed_by_bullseye': True, 'user_customized': False},
     'url': 'https://images-ssl.gotinder.com/5ca4c8f1e9a26316002be8be/1080x1350_e03a9f1f-6963-4fd6-88bf-5e3c05911eed.jpg',
     'processedFiles': [{'url': 'https://images-ssl.gotinder.com/5ca4c8f1e9a26316002be8be/640x800_e03a9f1f-6963-4fd6-88bf-5e3c05911eed.jpg'

In [21]:
recommendations['results'][0]['_id']

'5ca4c8f1e9a26316002be8be'

In [ ]:
!sqlitebiter -o sample.sqlite file *

# API

In [10]:
def get_auth_token(fb_auth_token, fb_user_id):
    if "error" in fb_auth_token:
        return {"error": "could not retrieve fb_auth_token"}
    if "error" in fb_user_id:
        return {"error": "could not retrieve fb_user_id"}
    url = url = 'https://api.gotinder.com/v2/auth/login/facebook'
    req = requests.post(url,
                        headers=headers,
                        data=json.dumps(
                            {'token': fb_auth_token, 'facebook_id': fb_user_id})
                        )
    print('req',req)
    
    #try:
    
    tinder_auth_token = req.json()['data']['api_token']
    print('token :', tinder_auth_token)
    headers.update({"X-Auth-Token": tinder_auth_token})
    print("You have been successfully authorized!")
    return tinder_auth_token
    
    #except Exception as e:
    print(e)
    return {"error": "Something went wrong. Sorry, but we could not authorize you."}


def authverif():
    res = get_auth_token(config.fb_access_token, config.fb_user_id)
    if "error" in res:
        return False
    return True


def get_recommendations():
    '''
    Returns a list of users that you can swipe on
    '''
    try:
        r = requests.get('https://api.gotinder.com/user/recs', headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong with getting recomendations:", e)


def get_updates(last_activity_date=""):
    '''
    Returns all updates since the given activity date.
    The last activity date is defaulted at the beginning of time.
    Format for last_activity_date: "2017-07-09T10:28:13.392Z"
    '''
    try:
        url = config.host + '/updates'
        r = requests.post(url,
                          headers=headers,
                          data=json.dumps({"last_activity_date": last_activity_date}))
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong with getting updates:", e)


def get_self():
    '''
    Returns your own profile data
    '''
    try:
        url = config.host + '/profile'
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your data:", e)


def change_preferences(**kwargs):
    '''
    ex: change_preferences(age_filter_min=30, gender=0)
    kwargs: a dictionary - whose keys become separate keyword arguments and the values become values of these arguments
    age_filter_min: 18..46
    age_filter_max: 22..55
    age_filter_min <= age_filter_max - 4
    gender: 0 == seeking males, 1 == seeking females
    distance_filter: 1..100
    discoverable: true | false
    {"photo_optimizer_enabled":false}
    '''
    try:
        url = config.host + '/profile'
        r = requests.post(url, headers=headers, data=json.dumps(kwargs))
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not change your preferences:", e)


def get_meta():
    '''
    Returns meta data on yourself. Including the following keys:
    ['globals', 'client_resources', 'versions', 'purchases',
    'status', 'groups', 'products', 'rating', 'tutorials',
    'travel', 'notifications', 'user']
    '''
    try:
        url = config.host + '/meta'
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your metadata:", e)

In [11]:
def get_meta_v2():
    '''
    Returns meta data on yourself from V2 API. Including the following keys:
    ['account', 'client_resources', 'plus_screen', 'boost',
    'fast_match', 'top_picks', 'paywall', 'merchandising', 'places',
    'typing_indicator', 'profile', 'recs']
    '''
    try:
        url = config.host + '/v2/meta'
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your metadata:", e)

def update_location(lat, lon):
    '''
    Updates your location to the given float inputs
    Note: Requires a passport / Tinder Plus
    '''
    try:
        url = config.host + '/passport/user/travel'
        r = requests.post(url, headers=headers, data=json.dumps({"lat": lat, "lon": lon}))
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not update your location:", e)

def reset_real_location():
    try:
        url = config.host + '/passport/user/reset'
        r = requests.post(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not update your location:", e)


def get_recs_v2():
    '''
    This works more consistently then the normal get_recommendations becuase it seeems to check new location
    '''
    try:
        url = config.host + '/v2/recs/core?locale=en-US'
        r = requests.get(url, headers=headers)
        return r.json()
    except Exception as e:
        print('excepted')

def set_webprofileusername(username):
    '''
    Sets the username for the webprofile: https://www.gotinder.com/@YOURUSERNAME
    '''
    try:
        url = config.host + '/profile/username'
        r = requests.put(url, headers=headers,
                         data=json.dumps({"username": username}))
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not set webprofile username:", e)

def reset_webprofileusername(username):
    '''
    Resets the username for the webprofile
    '''
    try:
        url = config.host + '/profile/username'
        r = requests.delete(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not delete webprofile username:", e)

def get_person(id):
    '''
    Gets a user's profile via their id
    '''
    try:
        url = config.host + '/user/%s' % id
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get that person:", e)


def send_msg(match_id, msg):
    try:
        url = config.host + '/user/matches/%s' % match_id
        r = requests.post(url, headers=headers,
                          data=json.dumps({"message": msg}))
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not send your message:", e)

def unmatch(match_id):
    try:
        url = config.host + '/user/matches/%s' % match_id
        r = requests.delete(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not unmatch person:", e)

def superlike(person_id):
    try:
        url = config.host + '/like/%s/super' % person_id
        r = requests.post(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not superlike:", e)


def like(person_id):
    try:
        url = config.host + '/like/%s' % person_id
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not like:", e)

#Theresia on Tinder

In [12]:
def dislike(person_id):
    try:
        url = config.host + '/pass/%s' % person_id
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not dislike:", e)


def report(person_id, cause, explanation=''):
    '''
    There are three options for cause:
        0 : Other and requires an explanation
        1 : Feels like spam and no explanation
        4 : Inappropriate Photos and no explanation
    '''
    try:
        url = config.host + '/report/%s' % person_id
        r = requests.post(url, headers=headers, data={
                          "cause": cause, "text": explanation})
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not report:", e)


def match_info(match_id):
    try:
        url = config.host + '/matches/%s' % match_id
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your match info:", e)

def all_matches():
    try:
        url = config.host + '/v2/matches'
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your match info:", e)

def fast_match_info():
    try:
        url = config.host + '/v2/fast-match/preview'
        r = requests.get(url, headers=headers)
        count = r.headers['fast-match-count']
      # image is in the response but its in hex..
        return count
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your fast-match count:", e)

def trending_gifs(limit=3):
    try:
        url = config.host + '/giphy/trending?limit=%s' % limit
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get the trending gifs:", e)

def gif_query(query, limit=3):
    try:
        url = config.host + '/giphy/search?limit=%s&query=%s' % (limit, query)
        r = requests.get(url, headers=headers)
        return r.json()
    except requests.exceptions.RequestException as e:
        print("Something went wrong. Could not get your gifs:", e)


# def see_friends():
#     try:
#         url = config.host + '/group/friends'
#         r = requests.get(url, headers=headers)
#         return r.json()['results']
#     except requests.exceptions.RequestException as e:
#         print("Something went wrong. Could not get your Facebook friends:", e)